# Applied Data Science Capstone
This notebook is dedicated to the final project of the Data science specialization on Coursera. 

In [33]:
# import libraries
import pandas as pd
import numpy as np

## Task 1

In [34]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Task 2 : import data form html
In this part, we import data from the location and Neighbourhood of Cananda. We import the data from the webiste https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

For this task, we use the libraries ``BeautifulSoup`` and ``Pandas``.

In [35]:
# import libraries for url data
from bs4 import BeautifulSoup
import requests

In [36]:
# specify url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# request html
page = requests.get(url)

soup = BeautifulSoup(page.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

We print the initial shape of the dataset.

In [37]:
df.shape

(287, 3)

In [38]:
# first 5 rows of the dataset
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We observe first that the data contains some missing values. These missing values are contained in the feature Borough and Neighbourhood. They are designed by 'Not assigned'. We decide first to remove the rows containing these missing values.

In [39]:
df_prep = df[df.Borough != 'Not assigned']
df_prep.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


The next step is to reindex the dataframe and we remove the old index.

In [40]:
df_prep_ind = df_prep.reset_index()
df_prep_ind.head()

,index,Postcode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor


In [41]:
df_reind = df_prep_ind.drop(['index'], axis=1)
df_reind.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


We observe that there are some redundant data. Since on Borough can be associated to at least two Neighourhood, then we need to combine each row with the same Borough and Postcode as one row. Then the feature Neighbourhood contains list with many neighbourhoods for one Borough and one Postcode.

In [46]:
# list of unique Postcode
postcode = np.unique(df_reind.Postcode)

In [12]:
# print the Neighbourhood of the Postcode M1B
df_reind[df_reind['Postcode'] == 'M1B']['Neighbourhood']

7      Rouge
8    Malvern
Name: Neighbourhood, dtype: object

In [18]:
# combine the two neighbourhoods for the Postcode M1B
df_reind.loc[df_reind['Postcode'] == 'M1B','Neighbourhood'].str.cat(sep=',')

'Rouge,Malvern'

In [47]:
# combine neighbourhood
for p in postcode:
    df_reind.loc[df_reind['Postcode'] == p,'Neighbourhood'] = df_reind.loc[df_reind['Postcode'] == p,'Neighbourhood'].str.cat(sep=',')

In [48]:
# print dataframe after cleaning
df_reind.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [49]:
# verify the Neighbourhood for one Postcode.
df_reind.loc[df_reind['Postcode'] == 'M9V']

,Postcode,Borough,Neighbourhood
173,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
174,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
175,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
176,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
177,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
178,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
179,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
180,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


We observe that we still have duplicate rows with Neighbourhood as list. Then we need to remove the redundant column.

In [50]:
# drop redundant columns
df_reind.drop_duplicates('Postcode', inplace=True)

In [51]:
# print the clean dataframe
df_reind.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,"Rouge,Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens,Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson,Garden District"


In [52]:
# print the shape of the clean data
df_reind.shape

(103, 3)

In order to not re-call each cell, we decide to save the clean dataframe as csv file.

In [53]:
# convert to csv
df_reind.to_csv('canada_bor.csv')

In [ ]:
# import the csv 
df = pd.read_csv('canada_bor.csv')

## Task 3: add geographic features
In this section, we add the coordinate of each Borough. To perform this task, we use the csv-file ``Geospatial_Coordinates.csv`` which contains the coordiante and the Postal code and merge with the previous data.

In [54]:
# import geo data
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In order to merge this dataset with the previous dataset, we need to rename the Postcode feature so that it coincide with the coordinate dataset and then we can merge the two dataset.

In [58]:
df_reind.rename({'Postcode': 'Postal Code'}, inplace=True, axis=1)
df_reind.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [59]:
df_reind.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [62]:
result = pd.merge(df_reind, coord, on='Postal Code')
result.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [63]:
# load final data to csv
result.to_csv('result.csv')

In [64]:
# shape of final data
result.shape

(103, 5)